In [ ]:
import os
import glob
import json
import time
import pickle
import tweepy

from datetime import datetime

import pandas as pd

In [ ]:
data_dir_path = '../data'

In [ ]:
key_dir_path = '../keys'

In [ ]:
key_paths = glob.glob(os.path.join(keys_path, '*'))
key_paths = [key.replace('\\', '/') for key in key_paths]

In [ ]:
username_path = os.path.join(data_path, 'supports', 'list_of_buzzer.csv')

In [ ]:
for username in d_buzzers.username.to_list()[:3]:
    username = username.replace('@', '')
    print(username)
    profile = get_profile_user(username)
    
    if type(profile) == tweepy.error.TweepError:
        user_raw_all.extend({username: profile})
        user_and_following_dict[username] = None
        username_has_collected.append(username)
    else:
        follower_list, user_raw = get_following(username)

        user_and_following_dict[profile.screen_name] = follower_list
        user_raw_all.extend([{user.screen_name: user} for user in user_raw])
        user_raw_all.extend({profile.screen_name: profile})
        username_has_collected.append(username)

In [ ]:
class Friends():
    def __init__(self, keys_paths, username_path):
        self.keys = self.read_key(keys_paths)
        self.apis = self.auth_twitter()
        self.apis_is_used = [False] * len(self.apis)
        self.usernames = self.read_csv(username_path)
        
    def read_key(self, key_paths):
        return [pickle.load(open(path, 'rb')) for path in key_paths]
    
    def read_csv(self, path):
        d_data = pd.read_csv(path)
        return d_data
    
    def auth_twitter(self):
        api_list = []
        for key in self.keys:
            auth = tweepy.OAuthHandler(key["api_key"], key["api_secret_key"])
            auth.set_access_token(key["access_token"], key["access_token_secret"])
            api = tweepy.API(auth)
            
            api_list.append(api)
            
        return api_list
    
    def output(data, path_dir, filename):
        pickle.dump(data, open(os.path.join(path_dir, filename), 'wb'))
        print("success", filename)
        
    def get_profile_user(self, username):
        while True:
            try:
                profile = api.get_user(username)
                return profile
            except tweepy.TweepError as e:
                err_msg = e
                return err_msg
            except tweepy.RateLimitError:
                print("\tRateLimit")
                print(datetime.today().strftime("\t%H:%M:%S %d-%m-%Y"))
                time.sleep(15*60)
    
    def limit_handled(self, cursor):
        while True:
            try:
                yield cursor.next()
            except tweepy.RateLimitError:
                print('\tRateLimit')
                print(datetime.today().strftime("\t%H:%M:%S %d-%m-%Y"))
                time.sleep(15 * 60)
            except:
                break
    
    def get_following(self, username):
        follower_list = []
        user_raw = []
        for follower in limit_handled(tweepy.Cursor(api.friends, id=username).items()):
            follower_list.append(follower.screen_name)
            user_raw.append(follower)

        return follower_list, user_raw

In [ ]:
friends = Friends(keys_path, username_path)